In [111]:
import json
import pandas as pd
import numpy as np
from pathlib import Path 
import re
import ast

In [112]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"
input_directory = Path(f'{main_path}TikTok/Input_test')  
print(input_directory)
max_columns = 7


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/TikTok/Input_test


In [113]:
for i in range(max_columns):
        col_path = [f"col_path_{i}" for i in range(max_columns)]
        col_path_values = [f"col_path_{i}_values" for i in range(1, max_columns)]
        col_path_list = [f"col_path_{i}_lIST" for i in range(1, max_columns)]

In [114]:
print(col_path_values)

['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values', 'col_path_6_values']


### loading_data()

In [115]:
def loading_data(data):
    

    with open(data, 'r') as file:
        data = json.load(file)
    
    # Check the first (top-level) key
    first_key = next(iter(data))

    # If it ends with `.json`, unwrap the inner content
    if first_key.endswith(".json"):
        data = data[first_key]  # remove outer key

    df = pd.DataFrame()
    df['col_path_0_values'] = data
    df['col_path_0'] = df.index
    
   

    df.reset_index(inplace = True)
    df = df.drop('index', axis = 1)


        

    for i in range(1,max_columns):   
        df[f'col_path_{i}'] = ''
        df[f'col_path_{i}_values'] = ''
        df[f'col_path_{i}_LIST'] = ''


    
    return(df)



### flatten_json()

In [116]:
def flatten_json(df):
    
    new_rows = []
    current_rows = df.copy()  # snapshot to iterate over

    for ix, row in current_rows.iterrows():
        value= row['col_path_0_values']
            
        if isinstance(value, list):
                    # Check if list of dicts
            if value and isinstance(value[0], dict):
                
                seen = set()
                unique = []
                for d in value:
                    # Use json.dumps to get a hashable, sorted representation
                    s = json.dumps(d, sort_keys=True)
                    if s not in seen:
                        seen.add(s)
                        unique.append(d)
                value = unique
                

                for item in value:
                    new_row = row.copy()
                    new_row['col_path_0_values'] = item
                    new_row['col_path_0_LIST'] = 'LIST'
                    new_rows.append(new_row)
        else:
             df.at[ix, 'col_path_0_LIST'] = 'NO LIST'
    



    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    
    new_rows.clear()

           

    new_rows = []

    for i in range(max_columns):
        current_rows = df.copy()  # snapshot to iterate over

        for ix, row in current_rows.iterrows():
            value = row.get(f'col_path_{i}_values', None)
    

            if isinstance(value, dict):
                for k, v in value.items():
                    if isinstance(v, list):
                        # Check if list of dicts
                        if v and isinstance(v[0], dict):

                            seen = set()
                            unique = []
                            for d in v:
                                # Use json.dumps to get a hashable, sorted representation
                                s = json.dumps(d, sort_keys=True)
                                if s not in seen:
                                    seen.add(s)
                                    unique.append(d)
                            v = unique
                            

                            for item in v:
                                new_row = row.copy()
                                new_row[f'col_path_{i+1}_values'] = item
                                new_row[f'col_path_{i+1}'] = k
                                new_row[f'col_path_{i+1}_LIST'] = 'LIST'
                                new_rows.append(new_row)
                        else:
                            # Regular list (e.g., strings, ints)
                            v = v[0]
                            new_row = row.copy()
                            new_row[f'col_path_{i+1}_values'] = v
                            new_row[f'col_path_{i+1}'] = k
                            new_row[f'col_path_{i+1}_LIST'] = 'LIST'
                            new_rows.append(new_row)
                    else:
                
                        new_row = row.copy()
                        new_row[f'col_path_{i+1}_values'] = v
                        new_row[f'col_path_{i+1}'] = k
                        new_row[f'col_path_{i+1}_LIST'] = 'NO LIST'
                        new_rows.append(new_row)
                

        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
        new_rows.clear()
            
                        


    

    df = df.replace(r'^\s*$', np.nan, regex=True)
    df_red = df.drop(columns=[col for col in df.columns if col.endswith("_LIST")])
    
    df_red['last_valid_index'] = df_red.apply(pd.Series.last_valid_index, axis=1)
    df_red = df_red.drop(columns=[col for col in df.columns if col.endswith("values")])

    
    col_path = [f"col_path_{i}" for i in range(max_columns)]
    df = pd.merge(df, df_red, on = col_path, how='left')
    
    
    return(df)

### process_col_path()
The 'process_col_path()' function checks whether the value in a row of for one of the column paths is actually a datatype and stores this value in a column data_type and replaces the original value with NA. The data types are the lowest level values in the JSON files

In [117]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object', 'str', 'int', 'float', 'bool', 'dict', 'list']



# Define the function 'process_col_path()'
def process_col_path(df, col_path_values, data_types):
    df['data_type'] = ''

    """
    row: Rows in the dataframe
    columns: List of column names of column path columns 
    data_types: List of the values that are data types
    """
    
    for ix, row in df.iterrows():
        for col in [f"col_path_{i}_values" for i in range(1, max_columns)]:
            

            #If the value stored in the column is found in the list 'data_types', 
            if row[col] in data_types:
                # this value is placed in the column 'data_type'
                value = row[col]
                
                df.at[ix, 'data_type'] = value
            else:
                continue
               
    
                
    return df

### remove_intermediate_rows()

In [118]:
def remove_intermediate_rows(df):
    drop_index = []
    for ix, row in df.iterrows():
            last_value = row[f"{row['last_valid_index']}"]
           
        
            if isinstance(last_value, str):
                if last_value in (data_types):
                        col =  row['last_valid_index']
                        col_level = re.findall(r'\d+', col)
                        col_level = col_level[0]
                                
                                
                        #df.at[ix, f"col_path_{col_level}_LIST"] = np.nan

                                
                else:
                        drop_index.append(ix)
            else:
                drop_index.append(ix)
                  
        
   
    df = df.drop(drop_index)
    df = df.drop(columns=[col for col in df.columns if col.endswith("values")])

    
    return df

### extract_path()

In [119]:
def extract_path(df, max_columns):
    
    df['path']= ''
    
    for ix, row in df.iterrows():
        path = []
    
        for col in [f"col_path_{i}" for i in range(max_columns)]:
            val = row[col]
        
            if pd.notna(val):

                path.append(str(val).strip())

      
        
        df.at[ix, 'path'] = path
       
    
        

    return df




### list_path()

In [120]:
def list_path(df):
    
    df['list_path'] = ''
    df['subfield_path'] = ''
    df['column_name'] = ''
    df['var_type'] = ''
    
    new_rows = []
    delete_rows = []

    for ix, row in df.iterrows():
        
        path = row['path'] 
        
        
        for i in range(len(path)):
           
            if row[f"col_path_{i}_LIST"] == 'LIST':
                
                new_row = row.copy()
                delete_rows.append(ix)
                new_row['var_type'] = 'list'
                new_row['list_path'] = path[:i]
                new_row['subfield_path'] = path[i:]
                new_row['column_name'] = path[-1]
                new_rows.append(new_row)
                break

            elif row[f"col_path_{i}_LIST"] == "NO LIST":
                var_type = 'static'
                subfield_path = path
                column_name = path[-1]

            elif row[f"col_path_{i}_LIST"] == 'nan':
                var_type = 'skip'
                subfield_path = path
                column_name = path[-1]
               
            else:
                var_type = 'skip'
                subfield_path = path
                column_name = path[-1]


           
        df.at[ix, 'subfield_path'] = subfield_path
        df.at[ix, 'column_name'] = column_name
        df.at[ix, 'var_type'] = var_type

    df = df.drop(index=delete_rows).reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    new_rows.clear()
    
        


    
      
        
    return df

### clean_and_store()
This function orders the columns in the DataFrame, resets the index, fills the NA and saves the DataFrame as CSV

In [121]:
def clean_and_store(df, file_name):

    """
    df: The dataframe that will be cleaned and stored
    file_name: The filename of data structure that is being processed
    """

  
   
    df=  df[df['col_path_0'] != 'Direct Message']
    
    
  
    df = df[['column_name', 'path', 'list_path', 'subfield_path', 'var_type', 'data_type']]
    # Save the DataFrame 
    df.to_csv(f"{main_path}TikTok/Output/Output_" + file_name + '.csv', index=False)

    return df

### structure_donations()
The structure_donations() function executes all functions above and results in a saved DataFrame for each data structure.

In [122]:
def structure_donations(data, col_path, max_columns):
     # Store the path to the data structure
    data = Path(data)  
    
    # Save teh file name of the data structure
    file_name = Path(data).stem 

    df = loading_data(data)
    print('FINISH: loading_data()')
  
    df = flatten_json(df)
    print('FINISH: flatten_json()')
    df = process_col_path(df, col_path_values, data_types)
    print('FINISH: process_col_path()')
    df = remove_intermediate_rows(df)
    print('FINISH: remove_intermediate_rows()')
    df = extract_path(df, max_columns)
    print('FINISH: extract_path()')
    df = list_path(df)
    print('FINISH: list_path()')

    display(df)
    df = clean_and_store(df, file_name)
    print('FINISH: clean_and_store()')

    return(df)
    
    

    

## Execute 'structure_donations()': Transform data structures from JSON format to tabular format

In [123]:
# Execute the 'structure_donations()' function for each file (data structure) in the input directory
for file in input_directory.iterdir():  
    if file.is_file():  
        print(file.name)
        structure_donations(file, col_path, max_columns)

HJ_tiktol_json_strcure.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()


/tmp/ipykernel_11592/2077604788.py:98: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


,col_path_0,col_path_1,col_path_1_LIST,col_path_2,col_path_2_LIST,col_path_3,col_path_3_LIST,col_path_4,col_path_4_LIST,col_path_5,...,col_path_6,col_path_6_LIST,col_path_0_LIST,last_valid_index,data_type,path,list_path,subfield_path,column_name,var_type
0,Ads and data,Ad Interests,NO LIST,AdInterestCategories,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,string,"[Ads and data, Ad Interests, AdInterestCategor...",,"[Ads and data, Ad Interests, AdInterestCategor...",AdInterestCategories,static
1,Ads and data,Instant Form Ads Responses,NO LIST,ResponsesList,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,object,"[Ads and data, Instant Form Ads Responses, Res...",,"[Ads and data, Instant Form Ads Responses, Res...",ResponsesList,static
2,Ads and data,Off TikTok Activity,NO LIST,OffTikTokActivityDataList,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,object,"[Ads and data, Off TikTok Activity, OffTikTokA...",,"[Ads and data, Off TikTok Activity, OffTikTokA...",OffTikTokActivityDataList,static
3,App Settings,Block List,NO LIST,App,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,number,"[App Settings, Block List, App]",,"[App Settings, Block List, App]",App,static
4,App Settings,Block List,NO LIST,BlockList,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,object,"[App Settings, Block List, BlockList]",,"[App Settings, Block List, BlockList]",BlockList,static
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Direct Message,Direct Messages,NO LIST,ChatHistory,NO LIST,Chat History with jannemertens99:,LIST,From,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Direct Message, Direct Messages, ChatHistory,...","[Direct Message, Direct Messages, ChatHistory]","[Chat History with jannemertens99:, From]",From,list
154,Direct Message,Direct Messages,NO LIST,ChatHistory,NO LIST,Chat History with jannemertens99:,LIST,Content,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Direct Message, Direct Messages, ChatHistory,...","[Direct Message, Direct Messages, ChatHistory]","[Chat History with jannemertens99:, Content]",Content,list
155,Direct Message,Direct Messages,NO LIST,ChatHistory,NO LIST,Chat History with nynkeveenstraa:,LIST,Date,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Direct Message, Direct Messages, ChatHistory,...","[Direct Message, Direct Messages, ChatHistory]","[Chat History with nynkeveenstraa:, Date]",Date,list
156,Direct Message,Direct Messages,NO LIST,ChatHistory,NO LIST,Chat History with nynkeveenstraa:,LIST,From,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Direct Message, Direct Messages, ChatHistory,...","[Direct Message, Direct Messages, ChatHistory]","[Chat History with nynkeveenstraa:, From]",From,list


FINISH: clean_and_store()
TT_structure_tiktok_takeout_ro.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()


/tmp/ipykernel_11592/2077604788.py:98: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


...

,col_path_0,col_path_1,col_path_1_LIST,col_path_2,col_path_2_LIST,col_path_3,col_path_3_LIST,col_path_4,col_path_4_LIST,col_path_5,...,col_path_6,col_path_6_LIST,col_path_0_LIST,last_valid_index,data_type,path,list_path,subfield_path,column_name,var_type
0,Ads and data,Ad Interests,NO LIST,AdInterestCategories,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,string,"[Ads and data, Ad Interests, AdInterestCategor...",,"[Ads and data, Ad Interests, AdInterestCategor...",AdInterestCategories,static
1,Ads and data,Instant Form Ads Responses,NO LIST,ResponsesList,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,object,"[Ads and data, Instant Form Ads Responses, Res...",,"[Ads and data, Instant Form Ads Responses, Res...",ResponsesList,static
2,Ads and data,Off TikTok Activity,NO LIST,OffTikTokActivityDataList,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,object,"[Ads and data, Off TikTok Activity, OffTikTokA...",,"[Ads and data, Off TikTok Activity, OffTikTokA...",OffTikTokActivityDataList,static
3,App Settings,Block List,NO LIST,App,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,number,"[App Settings, Block List, App]",,"[App Settings, Block List, App]",App,static
4,App Settings,Block List,NO LIST,BlockList,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,object,"[App Settings, Block List, BlockList]",,"[App Settings, Block List, BlockList]",BlockList,static
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Your Activity,Watch History,NO LIST,VideoList,LIST,Link,NO LIST,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_3_values,string,"[Your Activity, Watch History, VideoList, Link]","[Your Activity, Watch History]","[VideoList, Link]",Link,list
118,Profile,Profile Info,NO LIST,ProfileMap,NO LIST,PlatformInfo,LIST,Description,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Profile, Profile Info, ProfileMap, PlatformIn...","[Profile, Profile Info, ProfileMap]","[PlatformInfo, Description]",Description,list
119,Profile,Profile Info,NO LIST,ProfileMap,NO LIST,PlatformInfo,LIST,Name,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Profile, Profile Info, ProfileMap, PlatformIn...","[Profile, Profile Info, ProfileMap]","[PlatformInfo, Name]",Name,list
120,Profile,Profile Info,NO LIST,ProfileMap,NO LIST,PlatformInfo,LIST,Platform,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Profile, Profile Info, ProfileMap, PlatformIn...","[Profile, Profile Info, ProfileMap]","[PlatformInfo, Platform]",Platform,list


/tmp/ipykernel_11592/2077604788.py:98: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


FINISH: clean_and_store()
tiktok_takeout_dutch_structure.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()


,col_path_0,col_path_1,col_path_1_LIST,col_path_2,col_path_2_LIST,col_path_3,col_path_3_LIST,col_path_4,col_path_4_LIST,col_path_5,...,col_path_6,col_path_6_LIST,col_path_0_LIST,last_valid_index,data_type,path,list_path,subfield_path,column_name,var_type
0,Activity,Favorite Videos,NO LIST,App,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,number,"[Activity, Favorite Videos, App]",,"[Activity, Favorite Videos, App]",App,static
1,Activity,Follower List,NO LIST,App,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,number,"[Activity, Follower List, App]",,"[Activity, Follower List, App]",App,static
2,Activity,Follower List,NO LIST,IsFastLane,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,boolean,"[Activity, Follower List, IsFastLane]",,"[Activity, Follower List, IsFastLane]",IsFastLane,static
3,Activity,Following List,NO LIST,App,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,number,"[Activity, Following List, App]",,"[Activity, Following List, App]",App,static
4,Activity,Following List,NO LIST,IsFastLane,NO LIST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_2_values,boolean,"[Activity, Following List, IsFastLane]",,"[Activity, Following List, IsFastLane]",IsFastLane,static
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,Video,Videos,NO LIST,VideoList,LIST,AddYoursText,NO LIST,NaN,NaN,NaN,...,NaN,NaN,NO LIST,col_path_3_values,string,"[Video, Videos, VideoList, AddYoursText]","[Video, Videos]","[VideoList, AddYoursText]",AddYoursText,list
127,Profile,Profile Information,NO LIST,ProfileMap,NO LIST,PlatformInfo,LIST,Description,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Profile, Profile Information, ProfileMap, Pla...","[Profile, Profile Information, ProfileMap]","[PlatformInfo, Description]",Description,list
128,Profile,Profile Information,NO LIST,ProfileMap,NO LIST,PlatformInfo,LIST,Name,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Profile, Profile Information, ProfileMap, Pla...","[Profile, Profile Information, ProfileMap]","[PlatformInfo, Name]",Name,list
129,Profile,Profile Information,NO LIST,ProfileMap,NO LIST,PlatformInfo,LIST,Platform,NO LIST,NaN,...,NaN,NaN,NO LIST,col_path_4_values,string,"[Profile, Profile Information, ProfileMap, Pla...","[Profile, Profile Information, ProfileMap]","[PlatformInfo, Platform]",Platform,list


FINISH: clean_and_store()


## Merge all data structures into one schema_df

In [124]:
# Path to the folder containing CSV files
output_path = f"{main_path}TikTok/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

# Drop rows that are completely identical across all columns
merged_df = merged_df.drop_duplicates()
display(merged_df)


,column_name,path,list_path,subfield_path,var_type,data_type
0,App,"['Activity', 'Favorite Videos', 'App']",NaN,"['Activity', 'Favorite Videos', 'App']",static,number
1,App,"['Activity', 'Follower List', 'App']",NaN,"['Activity', 'Follower List', 'App']",static,number
2,IsFastLane,"['Activity', 'Follower List', 'IsFastLane']",NaN,"['Activity', 'Follower List', 'IsFastLane']",static,boolean
3,App,"['Activity', 'Following List', 'App']",NaN,"['Activity', 'Following List', 'App']",static,number
4,IsFastLane,"['Activity', 'Following List', 'IsFastLane']",NaN,"['Activity', 'Following List', 'IsFastLane']",static,boolean
...,...,...,...,...,...,...
913,AIGeneratedContent,"['Video', 'RecentlyDeletedPosts', 'PostList', ...","['Video', 'RecentlyDeletedPosts']","['PostList', 'AIGeneratedContent']",list,string
914,Sound,"['Video', 'RecentlyDeletedPosts', 'PostList', ...","['Video', 'RecentlyDeletedPosts']","['PostList', 'Sound']",list,string
915,Location,"['Video', 'RecentlyDeletedPosts', 'PostList', ...","['Video', 'RecentlyDeletedPosts']","['PostList', 'Location']",list,string
916,Title,"['Video', 'RecentlyDeletedPosts', 'PostList', ...","['Video', 'RecentlyDeletedPosts']","['PostList', 'Title']",list,string


### Id creation

In [125]:
id_df = merged_df[['path', 'column_name']]
id_df = id_df.drop_duplicates()
id_df[ 'id'] = ''



for n in range( max_columns):
    duplicates = id_df[id_df.duplicated(subset='id', keep=False)]

    for i, row in duplicates.iterrows():
        path = row['path']
        path =  ast.literal_eval(path)
        path_zero = path[0]
        path_rest = path[-n:]  # Varies each loop to attempt uniqueness

        if path_zero in path_rest:
            id_list = path_rest
        
        else:

            id_list = [path_zero] + path_rest   
            
        new_id = ':'.join(id_list)

        id_df.at[i, 'id'] = new_id  # Properly update the DataFrame



merged_df = pd.merge(merged_df, id_df, on = ['path', 'column_name'], how = 'left')

col = merged_df.pop('id') 
merged_df.insert(0, 'id', col) 
#merged_df['id']= merged_df['id'].fillna(merged_df['variable'])


#### ID duplicate flags

In [126]:
merged_df = merged_df.drop_duplicates()
dup = merged_df[merged_df.duplicated('id', keep= False)]
dup['duplicate_flag'] = 'Yes'
dup = dup[['id', 'duplicate_flag']]
merged_df = pd.merge(merged_df, dup, on = 'id', how = 'left')
merged_df['duplicate_flag'] = merged_df['duplicate_flag'].fillna(value = 'No')

/tmp/ipykernel_11592/29815283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dup['duplicate_flag'] = 'Yes'


### Clean and Save

In [127]:
merged_df['json_name'] = ''

keep_columns = ['id', 'column_name', 
                 'path', 'list_path', 'subfield_path', 
                 'var_type', 'data_type', 'json_name', 'duplicate_flag']


merged_df = merged_df[keep_columns ]
merged_df = merged_df.astype(str)
merged_df = merged_df.drop_duplicates()

# Save the final merged DataFrame
merged_df.to_csv(f"{main_path}TikTok/Final/TT_Merged_structures.csv", index=False)